Below, I install my own implementation of Professor Boonstra's "memoize DataFrame to disk" feature. The source code can be found at [github.com/ethho/memoize](https://github.com/ethho/memoize).

In [1]:
!python3 -m pip install git+https://github.com/ethho/memoize.git

  Cloning https://github.com/ethho/memoize.git to /tmp/pip-req-build-f8yitl13
  Running command git clone --filter=blob:none --quiet https://github.com/ethho/memoize.git /tmp/pip-req-build-f8yitl13
  Resolved https://github.com/ethho/memoize.git to commit bef633bd22e4acde44cccb63399a176c6cef79b9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import json
import re
import os
from glob import glob
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import norm, probplot
import quandl
import functools
import plotly.express as px
import plotly.graph_objects as go
from src.ubacktester import (
    BacktestEngine, StrategyBase, PositionBase, FeedBase,
    PlotlyPlotter, FeedID, PriceFeed, px_plot
)
from memoize.dataframe import memoize_df

%matplotlib inline
pd.options.display.float_format = '{:,.4f}'.format

# 20230126_hw3_ho_ethan_12350006

@mpcs
@finm33550

Ethan Ho 1/20/2023

----


## Configuration & Helper Functions

The following cell contains helper functions and configuration options that I will use in this notebook.

In [3]:
def get_secrets(fp='./secrets.json'):
    """
    Reads secret values such as API keys from a JSON-formatted file at `fp`.
    """
    with open(fp, 'r') as f:
        data = json.load(f)
    return data

def get_quandl_api_key() -> str:
    """
    Returns Quandl API key stored in secrets.json.
    """
    secrets = get_secrets()
    key = secrets.get('NASTAQ_DATA_API_KEY')
    assert key, f"NASTAQ_DATA_API_KEY field in secrets.json is empty or does not exist"
    return key

def strip_str_dtypes(df: pd.DataFrame) -> pd.DataFrame:
    """
    Given a DataFrame, strips values in columns with string or object
    dtype. I noticed that this was an issue when I saw some m_ticker values
    like "AAPL       " with trailing whitespace.
    """
    for col in df.columns:
        if pd.api.types.is_string_dtype(df[col]) or pd.api.types.is_object_dtype(df[col]):
            df[col] = df[col].str.strip()
    return df

@memoize_df(cache_dir='/tmp/memoize')
def fetch_quandl_quotemedia_prices(
    start_date, end_date, ticker
) -> pd.DataFrame:
    df = quandl.get_table(
        'QUOTEMEDIA/PRICES',
        date={'gte': start_date, 'lte': end_date},
        ticker=ticker,
        api_key=get_quandl_api_key(),
    )
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by='date', inplace=True)
    return df

@memoize_df(cache_dir='/tmp/memoize')
def fetch_quandl_tbill_prices(
    start_date, end_date,
) -> pd.DataFrame:
    """Fetch table of treasury bill prices from Quandl."""
    df = quandl.get(
        ['USTREASURY/BILLRATES'],
        returns="pandas",
        start_date=start_date,
        end_date=end_date,
        ticker=ticker,
        api_key=get_quandl_api_key(),
    )
    df = df.reset_index().rename(columns={'Date': 'date'})
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by='date', inplace=True)
    return df

def risk_free_rate(**kw) -> float:
    """Calculates risk-free rate R_f from the 3-month T-bill rate."""
    tbill_prices = fetch_quandl_tbill_prices(**kw)
    tbill_returns = tbill_prices['USTREASURY/BILLRATES - 13 Wk Coupon Equiv']
    return tbill_returns.mean()

# Fetch Data Tables from Quandl

First, let's fetch the Zacks Fundmentals B. I chose to download zip archives as documented in the [Quandl API docs](https://github.com/quandl/quandl-python/blob/master/FOR_DEVELOPERS.md#datatable).

In [4]:
def fetch_zfb_table(table: str, include_cols: Optional[List] = None) -> pd.DataFrame:
    table_lower = table.lower()
    zip_fp = f'data/zacks_{table.lower()}.zip'
    if os.path.isfile(zip_fp):
        print(f"Zip file {zip_fp} already exists. Skipping download")
    else:
        export_table_kwargs = dict(
            filename=zip_fp,
            api_key=get_quandl_api_key(),
        )
        if table not in ('MT', 'HDM'):
            export_table_kwargs['per_end_date'] = {
                'gte': "2015-01-01", 'lte': "2022-01-01"
            }
        if include_cols:
            export_table_kwargs['qopts'] = {'columns': include_cols}
        fp = quandl.export_table(f'ZACKS/{table}',**export_table_kwargs)
        print(f"Wrote ZIP file to {zip_fp}")
    ! unzip -o -d data/zacks_{table_lower} {zip_fp}

    # DataFrame preprocessing
    df = pd.read_csv(*glob(f'data/zacks_{table_lower}/*.csv'))
    df = strip_str_dtypes(df)

    # Set MultiIndex of date and m_ticker
    if 'per_end_date' in df.columns:
        df['per_end_date'] = pd.to_datetime(df['per_end_date'])
        df.set_index(['m_ticker', 'per_end_date'], inplace=True)
        df.sort_index(level=(0, 1), inplace=True)
    else:
        df.set_index('m_ticker', inplace=True)
        df.sort_index(inplace=True)
    
    return df

In [5]:
! mkdir -p data

fc = fetch_zfb_table('FC', include_cols=[
    'ticker',
    'm_ticker',
    'per_end_date',
    'zacks_x_ind_code',
    'zacks_sector_code',
    'eps_diluted_net',
    'basic_net_eps',
    'net_lterm_debt',
    'tot_lterm_debt',
])

fr = fetch_zfb_table('FR', include_cols=[
    'ticker',
    'm_ticker',
    'per_end_date',
    'ret_invst',
    'tot_debt_tot_equity',
])

# fetch_zfb_table('MT', include_cols=[])

# fetch_zfb_table('HDM', include_cols=[])

mktv = fetch_zfb_table('MKTV', include_cols=[
    'ticker',
    'm_ticker',
    'per_end_date',
    'mkt_val',
])

shrs = fetch_zfb_table('SHRS', include_cols=[
    'ticker',
    'm_ticker',
    'per_end_date',
    'shares_out',
])

Zip file data/zacks_fc.zip already exists. Skipping download
Archive:  data/zacks_fc.zip
  inflating: data/zacks_fc/ZACKS_FC_2_6cedbbbdb119f11e23796711e30d7f51.csv  
Zip file data/zacks_fr.zip already exists. Skipping download
Archive:  data/zacks_fr.zip
  inflating: data/zacks_fr/ZACKS_FR_2_6e8c1a56149de6e1b305383b7d20f1d6.csv  
Zip file data/zacks_mktv.zip already exists. Skipping download
Archive:  data/zacks_mktv.zip
  inflating: data/zacks_mktv/ZACKS_MKTV_2_5f438acf10a403d4a32a49b0a3d10dc1.csv  
Zip file data/zacks_shrs.zip already exists. Skipping download
Archive:  data/zacks_shrs.zip
  inflating: data/zacks_shrs/ZACKS_SHRS_2_601b295228d7af010dc9a277c9f4d535.csv  


The next command will check file sizes, to make sure that we're not occupying too much disk space on graders' machines.

In [6]:
! du -hs data/zacks_*

12M	data/zacks_fc
2.8M	data/zacks_fc.zip
8.4M	data/zacks_fr
2.3M	data/zacks_fr.zip
13M	data/zacks_mktv
1.9M	data/zacks_mktv.zip
12M	data/zacks_shrs
1.5M	data/zacks_shrs.zip


# From HW 3 Prompt

Find $\ge$ 200 tickers where the following conditions are met for our analysis period of 1/2015 - 1/2022:

- Not in automotive, financial, or insurance sector at any point in the period
    - See `FC/ZACKS_X_IND_CODE`, `FC/ZACKS_SECTOR_CODE`, and the [classification list](http://www.zacksdata.com/app/download/247340904/Zacks+Sector+Industry+Mapping+Scheme.pdf) (and maybe `FC/ZACKS_METRICS_IND_CODE` too?)
- Debt/market cap ratio is $>0.1$ for some nontrivial amount of time. Should be about 1000-2000 companies including ASH, VTOL, ISUN, and VIVO.
- Calculation of the following ratios is feasible:
    - Debt to market cap
        - See `FR/TOT_DEBT_TOT_EQUITY`
    - Return on investment (ROI)
        - See `FR/RET_INVST`, `MKTV/MKT_VAL`, `FC/NET_LTERM_DEBT`, `FC/TOT_LTERM_DEBT`
    - Price to earnings (P/E)
        - See `FC/EPS_DILUTED_NET`, `FC/BASIC_NET_EPS`, `SHRS/SHARES_OUT`, `MKTV/MKT_VAL`, `GAAP`
        
# To-Do

- Strip string dtypes
- ffill trading day to `per_end_date` from previous trading day
- Narrow data query to only the columns we use below

# Data Munging to Get List of Tickers

From the FC table, let's filter out tickers in the excluded sectors. By the way, most of the time, when I refer to ticker, I'm actually referring to the `m_ticker`, which is easier to track historically as securities switch between exchanges.

In [7]:
fc['zacks_sector_code'] = fc['zacks_sector_code'].fillna(-1.).astype(int)
fc['zacks_x_ind_code'] = fc['zacks_x_ind_code'].fillna(-1.).astype(int)
fc.head()

ticker  zacks_x_ind_code  zacks_sector_code  \
m_ticker per_end_date                                               
NaN      2015-01-31     T.NA                12                 13   
         2015-04-30     T.NA                12                 13   
         2015-07-31     T.NA                12                 13   
         2015-10-31     T.NA                12                 13   
         2015-10-31     T.NA                12                 13   

                       eps_diluted_net  basic_net_eps  net_lterm_debt  \
m_ticker per_end_date                                                   
NaN      2015-01-31                NaN            NaN             NaN   
         2015-04-30                NaN            NaN             NaN   
         2015-07-31                NaN            NaN             NaN   
         2015-10-31                NaN            NaN             NaN   
         2015-10-31                NaN            NaN             NaN   

                       tot_lterm_debt  
m_ticker per_end_date                  
NaN      2015-01-31               NaN  
         2015-04-30               NaN  
         2015-07-31               NaN  
         2015-10-31               NaN  
         2015-10-31               NaN

We define which sectors we want to filter out for each column:

In [8]:
exclude_codes_spec = {
    'zacks_sector_code': (
        5, # automotive
        13, # finance
    ),
    'zacks_x_ind_code': [
        7, 8, 9, 10, 11, 210, # automotive
        *range(61, 70), # finance
        *range(85, 90), # insurance
    ]
}

In [9]:
print(f"There are {fc.index.get_level_values(0).unique().size} unique tickers before sector filtering.")

There are 11699 unique tickers before sector filtering.


We'll define a boolean mask and use it to get the list of `m_ticker`s we want to exclude.

In [10]:
fc['exclude_sector'] = (
    fc['zacks_sector_code'].isin(exclude_codes_spec['zacks_sector_code']) |
    fc['zacks_x_ind_code'].isin(exclude_codes_spec['zacks_x_ind_code'])
)
excluded_m_ticker = fc[fc['exclude_sector']].index.get_level_values(0).unique().tolist()
excluded_m_ticker[:10]

[nan, '#AQN', '#AVT', '#AZL', '#BLR', '#CED', '#EXH', '#FCO', '#FRP', '#INT']

In [11]:
fc.drop(index=excluded_m_ticker, inplace=True, errors='ignore')
print(f"There are {len(excluded_m_ticker)} tickers that are excluded due to sector.")

There are 2174 tickers that are excluded due to sector.


We removed about $\frac{1}{5}$ of the tickers with this filter.

Next, we'll filter to tickers where debt/market cap $> 1$. To do this, we need to merge in `FR/TOT_DEBT_TOT_EQUITY` and `MKTV/MKT_VAL`. As a first step, let's remove tickers where either of these values are null for all timepoints.

In [37]:
mkt_val_isnull_mask = mktv.groupby(level=0, group_keys=False).apply(lambda x: x['mkt_val'].isnull().all())
missing_mkt_val = mkt_val_isnull_mask[mkt_val_isnull_mask].index.tolist()
tdte_isnull_mask = fr.groupby(level=0, group_keys=False).apply(lambda x: x['tot_debt_tot_equity'].isnull().all())
missing_tdte = tdte_isnull_mask[tdte_isnull_mask].index.tolist()

# Drop m_tickers that are in excluded sector and
# dont' have mkt_val data
mktv.drop(index=excluded_m_ticker, inplace=True, errors='ignore')
mktv.drop(index=missing_mkt_val, inplace=True, errors='ignore')
mktv.drop(index=missing_tdte, inplace=True, errors='ignore')

In [38]:
print(f"There are {fr.index.get_level_values(0).unique().size} unique tickers in the MKTV table.")

There are 8075 unique tickers in the MKTV table.


In [39]:
fr.drop(index=excluded_m_ticker, inplace=True, errors='ignore')
fr.drop(index=missing_mkt_val, inplace=True, errors='ignore')
fr.drop(index=missing_tdte, inplace=True, errors='ignore')

In [40]:
print(f"There are {fr.index.get_level_values(0).unique().size} unique tickers in the FR table.")

There are 8075 unique tickers in the FR table.


In [48]:
fr_mktv = fr.merge(mktv['mkt_val'], how='outer', left_index=True, right_index=True)
fr_mktv['debt2cap'] = fr_mktv['tot_debt_tot_equity'] / fr_mktv['mkt_val']

In [56]:
fr_mktv.loc['#AAO']

,ticker,ret_invst,tot_debt_tot_equity,mkt_val,debt2cap
per_end_date,,,,,
2015-03-31,AAON,4.6118,NaN,"1,328.3200",NaN
2015-06-30,AAON,5.9382,NaN,"1,219.1700",NaN
2015-09-30,AAON,6.6799,NaN,"1,051.0000",NaN
2015-12-31,AAON,25.5581,NaN,"1,256.0400",NaN
2015-12-31,AAON,7.2368,NaN,"1,256.0400",NaN
2016-03-31,AAON,6.1223,NaN,"1,485.9300",NaN
2016-06-30,AAON,7.5519,NaN,"1,458.6600",NaN
2016-09-30,AAON,7.6581,NaN,"1,525.3000",NaN
2016-12-31,AAON,25.9235,NaN,"1,743.5500",NaN


In [54]:
fr_mktv.groupby(level=0, group_keys=False).apply(lambda x: (x['debt2cap'] > 0.1)).loc['ASH']

per_end_date
2015-03-31    False
2015-06-30    False
2015-09-30    False
2015-09-30    False
2015-12-31    False
2016-03-31    False
2016-06-30    False
2016-09-30    False
2016-09-30    False
2016-12-31    False
2017-03-31    False
2017-06-30    False
2017-09-30    False
2017-09-30    False
2017-12-31    False
2018-03-31    False
2018-06-30    False
2018-09-30    False
2018-09-30    False
2018-12-31    False
2019-03-31    False
2019-06-30    False
2019-09-30    False
2019-09-30    False
2019-12-31    False
2020-03-31    False
2020-06-30    False
2020-09-30    False
2020-09-30    False
2020-12-31    False
2021-03-31    False
2021-06-30    False
2021-09-30    False
2021-09-30    False
2021-12-31    False
Name: debt2cap, dtype: bool

In [16]:
fr_mkt = fr.merge(mktv['mkt_val'], how='outer', left_index=True, right_index=True)
fr_mkt

ticker  ret_invst  tot_debt_tot_equity    mkt_val
m_ticker per_end_date                                                  
#AAO     2015-03-31     AAON     4.6118                  NaN 1,328.3200
         2015-06-30     AAON     5.9382                  NaN 1,219.1700
         2015-09-30     AAON     6.6799                  NaN 1,051.0000
         2015-12-31     AAON    25.5581                  NaN 1,256.0400
         2015-12-31     AAON     7.2368                  NaN 1,256.0400
...                      ...        ...                  ...        ...
ZZ       2020-12-31      NaN        NaN                  NaN        NaN
         2021-03-31      NaN        NaN                  NaN        NaN
         2021-06-30      NaN        NaN                  NaN        NaN
         2021-09-30      NaN        NaN                  NaN        NaN
         2021-12-31      NaN        NaN                  NaN        NaN

[459788 rows x 4 columns]

In [17]:
def idx_diff(df1, df2):
    return set(df1.index.get_level_values(1).unique()).difference(df2.index.get_level_values(1).unique())

In [18]:
mktv[mktv.ticker == 'AAPL'].index

MultiIndex([('AAPL', '2015-03-31'),
            ('AAPL', '2015-06-30'),
            ('AAPL', '2015-09-30'),
            ('AAPL', '2015-12-31'),
            ('AAPL', '2016-03-31'),
            ('AAPL', '2016-06-30'),
            ('AAPL', '2016-09-30'),
            ('AAPL', '2016-12-31'),
            ('AAPL', '2017-03-31'),
            ('AAPL', '2017-06-30'),
            ('AAPL', '2017-09-30'),
            ('AAPL', '2017-12-31'),
            ('AAPL', '2018-03-31'),
            ('AAPL', '2018-06-30'),
            ('AAPL', '2018-09-30'),
            ('AAPL', '2018-12-31'),
            ('AAPL', '2019-03-31'),
            ('AAPL', '2019-06-30'),
            ('AAPL', '2019-09-30'),
            ('AAPL', '2019-12-31'),
            ('AAPL', '2020-03-31'),
            ('AAPL', '2020-06-30'),
            ('AAPL', '2020-09-30'),
            ('AAPL', '2020-12-31'),
            ('AAPL', '2021-03-31'),
            ('AAPL', '2021-06-30'),
            ('AAPL', '2021-09-30'),
            ('AAPL', '2021-1

In [19]:
fr_mkt.loc['AAPL']

,ticker,ret_invst,tot_debt_tot_equity,mkt_val
per_end_date,,,,
2015-03-31,AAPL,8.0253,0.3401,"729,926.3100"
2015-06-30,AAPL,6.1683,0.4330,"715,263.6900"
2015-09-30,AAPL,30.9201,0.5390,"629,010.0600"
2015-09-30,AAPL,6.4418,0.5390,"629,010.0600"
2015-12-31,AAPL,10.1179,0.4909,"583,613.0000"
2016-03-31,AAPL,5.2624,0.6122,"597,096.2500"
2016-06-30,AAPL,3.9881,0.6712,"515,134.8400"
2016-09-30,AAPL,22.4312,0.6786,"609,163.1300"
2016-09-30,AAPL,4.4257,0.6786,"609,163.1300"
